# RAG Implementation with Dorna-Llama3-8B-Instruct-Quantized

This notebook implements a Retrieval-Augmented Generation (RAG) system using:
- ChromaDB for vector storage and retrieval
- Sentence Transformer for embedding generation
- Llama3 for text generation

## 1. Import Dependencies

In [1]:
import sys
import time
import random
import chromadb
from llama_cpp import Llama
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

/home/amyrmahdy/GitHub/chatbot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Load Sample Data

In [2]:
# Define knowledge base chunks about NovaCloud service
context_data = {
    "services": '''
شرکت نواکلود سه سرویس اصلی ارائه می‌دهد:  
1. نوااستورج (NovaStorage) – یک سرویس ذخیره‌سازی ابری که برای شرکت‌های بزرگ طراحی شده است و امکان رمزگذاری سرتاسری (E2EE) و پشتیبان‌گیری خودکار را دارد.  
2. نواکامپیوتر (NovaCompute) – یک سرویس پردازش ابری که از پردازنده‌های ZetaCore X200 استفاده می‌کند و برای مدل‌های یادگیری ماشین سنگین بهینه‌سازی شده است.  
3. نواکانکت (NovaConnect) – یک پلتفرم شبکه خصوصی ابری (VPC) که به شرکت‌ها امکان ایجاد زیرشبکه‌های ایزوله با IP ثابت خصوصی را می‌دهد.  
''',
    "pricing": '''
نواکلود سه طرح قیمت‌گذاری ارائه می‌دهد:  
- طرح پایه (Basic): شامل ۵۰ گیگابایت فضای ذخیره‌سازی و ۲ هسته پردازشی با هزینه‌ی ۱۵ دلار در ماه  
- طرح حرفه‌ای (Pro): شامل ۵۰۰ گیگابایت فضای ذخیره‌سازی، ۸ هسته پردازشی و ترافیک نامحدود با هزینه‌ی ۶۰ دلار در ماه  
- طرح سازمانی (Enterprise): شامل ۵ ترابایت فضای ذخیره‌سازی، ۳۲ هسته پردازشی، و قابلیت تنظیم شبکه خصوصی اختصاصی با هزینه‌ی ۲۰۰ دلار در ماه  
''',
    "security": '''
نواکلود امنیت داده‌ها را با سه مکانیزم کلیدی تضمین می‌کند:  
- رمزگذاری سرتاسری (E2EE) برای داده‌های ذخیره‌شده در NovaStorage  
- احراز هویت چندمرحله‌ای (MFA) برای ورود به تمامی سرویس‌ها  
- فایروال هوشمند که تنها IPهای تأیید‌شده را به شبکه NovaConnect متصل می‌کند  
''',
    "uptime": '''
در سه ماه گذشته، NovaCompute در ۹۸.۹٪ مواقع بدون اختلال کار کرده است، اما یک قطعی ۲ ساعته در تاریخ ۱۵ فوریه ۲۰۲۴ به دلیل بروزرسانی سخت‌افزاری رخ داده است. در همین مدت، NovaStorage بدون هیچ اختلالی فعال بوده است.  
'''
}

# Convert dictionary to list of chunks for embedding
chunks = list(context_data.values())

## 3. Configure Embedding Model

In [3]:
# Load embedding model
def load_embedding_model(model_name='all-MiniLM-L6-v2'):
    """Load and configure the sentence transformer model for embeddings"""
    embedding_model_path = f"./models/{model_name}"
    
    # Load model from local path
    embedding_model = SentenceTransformer(embedding_model_path)
    
    # Create embedding function for ChromaDB
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name=model_name
    )
    
    return embedding_model, sentence_transformer_ef

# Initialize models
embedding_model, sentence_transformer_ef = load_embedding_model()

## 4. Set Up Vector Database

In [4]:
def setup_vector_db(collection_name="novacloud_knowledge", embedding_function=None):
    """Initialize ChromaDB and create or get collection"""
    client = chromadb.PersistentClient(path="./chromadb")
    
    # Get or create collection
    collection = client.get_or_create_collection(
        name=collection_name,
        metadata={"hnsw:space": "cosine"},  # Use cosine similarity for matching
        embedding_function=embedding_function
    )
    
    return client, collection

# Set up ChromaDB
chroma_client, collection = setup_vector_db(embedding_function=sentence_transformer_ef)

## 5. Add Documents to Vector Database

In [5]:
def add_documents_to_collection(collection, documents, embedding_model):
    """Add documents to ChromaDB collection with embeddings"""
    # Generate embeddings for documents
    embeddings = embedding_model.encode(documents)
    
    # Add documents with embeddings to collection
    collection.add(
        embeddings=embeddings,
        documents=documents,
        ids=[str(i) for i in range(len(documents))]
    )
    
    return len(documents)

# Add documents to collection
num_added = add_documents_to_collection(collection, chunks, embedding_model)
print(f"Added {num_added} documents to vector database")

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2

Added 4 documents to vector database


## 6. Implement Retrieval Function

In [6]:
def retrieve_relevant_documents(query, collection, embedding_model, top_k=1, similarity_threshold=0.7):
    """Retrieve relevant documents based on semantic similarity"""
    # Create embedding for query
    query_embedding = embedding_model.encode([query])
    
    # Query the collection
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k
    )
    
    # Extract results
    documents = results["documents"][0] if results["documents"] else ["No relevant information found."]
    distances = results["distances"][0] if results["distances"] else [1.0]  # Higher distance = less relevant
    
    # Print similarity scores for debugging
    print(f"Similarity scores: {[1-d for d in distances]}")
    
    # Optional: Filter by similarity threshold
    # filtered_docs = [doc for doc, dist in zip(documents, distances) if 1-dist >= similarity_threshold]
    # return filtered_docs if filtered_docs else ["No sufficiently relevant information found."]
    
    return documents

# Test retrieval function
test_query = "کدام سرویس نواکلود برای ذخیره‌سازی ابری استفاده می‌شود؟"
retrieved_docs = retrieve_relevant_documents(test_query, collection, embedding_model)
print(f"Retrieved document: {retrieved_docs[0][:100]}...")

Similarity scores: [0.5962998214527002]
Retrieved document: 
نواکلود امنیت داده‌ها را با سه مکانیزم کلیدی تضمین می‌کند:  
- رمزگذاری سرتاسری (E2EE) برای داده‌ها...


## 7. Load LLM for Generation

In [7]:
def load_llm_model(model_path="./models/Dorna-Llama3-8B-Instruct-GGUF-Q8/dorna-llama3-8b-instruct.Q8_0.gguf"):
    """Load and configure the LLM for text generation"""
    llm = Llama(
        model_path=model_path,
        chat_format="llama-3",
        n_gpu_layers=-1,  # Use all available GPU layers
        n_ctx=2048,       # Context window size
        verbose=False
    )
    
    return llm

# Load Llama model
llm = load_llm_model()

## 8. Create RAG Pipeline

In [8]:
# Define prompt template
PROMPT_TEMPLATE = '''
تاریخچه مکالمات:
{history}

دانش پایه:
{context}

سوال کاربر:
{prompt}
'''

# Initialize conversation history
conversation_history = []

def summarize_query(query,llm=llm):
    """Summarize query using the LLM"""
    messages = [
        {"role": "system", "content": "تو یک دستیار هستی که وظیفه تو خلاصه کردن متن است. سعی نکن به کاربر جواب بدی فقط تشریفات رو از بین ببر و تا جایی که امکان داره خلاصه کن. فقط تا جایی پرامپت کاربر رو کوتاه کن که به هسته اصلی مطلب آسیبی وارد نشه."},
        {"role": "user", "content": query}
    ]
    
    response = llm.create_chat_completion(
        messages=messages,
        top_p=0.85,
        temperature=0.1  # Low temperature for more deterministic responses
    )
    print(f"Summarized version: {response['choices'][0]['message']['content']}")
    return response['choices'][0]['message']['content']


def retrieve_context(query, collection=collection, embedding_model=embedding_model):
    """Retrieve relevant context based on the query"""
    docs = retrieve_relevant_documents(query, collection, embedding_model)
    return "\n".join(docs)

def generate_response(model_input, llm=llm):
    """Generate response using the LLM"""
    messages = [
        {"role": "system", "content": "تو یک دستیار متخصص و پشتیبانی فنی وضعیت سرویس ها هستی که با توجه به دانش پایه، به کاربر پاسخ کمک کننده میدی"},
        {"role": "user", "content": f"{model_input}"}
    ]
    
    response = llm.create_chat_completion(
        messages=messages,
        top_p=0.85,
        temperature=0.1  # Low temperature for more deterministic responses
    )
    
    return response['choices'][0]['message']['content']

def rag_chat(user_query, history=None):
    """Complete RAG pipeline: Retrieve → Generate → Respond"""
    if history is None:
        history = conversation_history
    
    # Summarize input query 
    # user_query_summarized = summarize_query(user_query)

    # Retrieve relevant context
    context = retrieve_context(user_query)
    
    # Format conversation history
    history_text = "\n".join(history)
    
    # Create prompt with context and history
    prompt = PROMPT_TEMPLATE.format(
        history=history_text,
        context=context, 
        prompt=user_query
    )
    
    # Generate response
    response = generate_response(prompt)
    
    # Summarize ouput query 
    # response_summarized = summarize_query(response)


    # Update conversation history
    history.append(f"سوال کاربر: {user_query}")
    history.append(f"پاسخ کمک کننده: {response}")
    
    return response

## 9. Text Streaming Utility

In [9]:
def stream_text(text, min_delay=0.02, max_delay=0.08):
    """Display text in a streaming manner, character by character"""
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        
        # Dynamic delay for natural effect
        time.sleep(random.uniform(min_delay, max_delay))
    
    print()  # New line after completion

## 10. Test RAG System

In [10]:
# Example 1: Basic question
query1 = "کدام سرویس نواکلود برای ذخیره‌سازی ابری استفاده می‌شود؟"
print(f"User query: {query1}")

# Reset conversation history
conversation_history = []

# Time the response
start = time.time()
response = rag_chat(query1)
end = time.time()

# Stream or print the response
stream_option = input("Stream response? (y/n): ").lower().strip() == 'y'
if stream_option:
    stream_text(response)
else:
    print(response)

print(f"---\nProcessing time: {end - start:.2f} seconds")

User query: کدام سرویس نواکلود برای ذخیره‌سازی ابری استفاده می‌شود؟
Similarity scores: [0.5962998214527002]
بر اساس دانش پایه، سرویس NovaStorage برای ذخیره‌سازی ابری استفاده می‌شود. این سرویس با استفاده از رمزگذاری سرتاسری (E2EE) برای داده‌های ذخیره‌شده تضمین می‌کند.
---
Processing time: 1.60 seconds


In [11]:
# Example 2: Follow-up question
query2 = "میشه بیشتر راجع به این سرویس توضیح بدی؟"
print(f"User query: {query2}")

# Time the response (using existing conversation history)
start = time.time()
response = rag_chat(query2)
end = time.time()

# Stream or print the response
if stream_option:
    stream_text(response)
else:
    print(response)

print(f"---\nProcessing time: {end - start:.2f} seconds")

User query: میشه بیشتر راجع به این سرویس توضیح بدی؟
Similarity scores: [0.608187841441516]
با خوشحالی! به عنوان یک دستیار متخصص و پشتیبانی فنی، خوشحالم که می‌توانم به شما در مورد سرویس NovaStorage توضیح بدهم.

نواستورج (NovaStorage) یک سرویس ذخیره‌سازی ابری است که توسط نواکلود ارائه می‌شود. این سرویس با استفاده از رمزگذاری سرتاسری (E2EE) برای داده‌های ذخیره‌شده تضمین می‌کند که داده‌های شما در دسترس دیگران قرار نخواهد گرفت. این رمزگذاری سرتاسری تضمین می‌کند که تنها شما و کسانی که به داده‌های شما دسترسی دارند، می‌توانند به آن‌ها دسترسی داشته باشند.

نواستورج همچنین از یک سیستم احراز هویت چندمرحله‌ای (MFA) برای ورود به تمامی سرویس‌ها استفاده می‌کند. این سیستم تضمین می‌کند که تنها کسانی که به داده‌های شما دسترسی دارند، می‌توانند به سرویس‌ها دسترسی داشته باشند.

از سوی دیگر، فایروال هوشمند که در شبکه NovaConnect استفاده می‌شود، تنها IPهای تأیید‌شده را به شبکه متصل می‌کند. این تضمین می‌کند که تنها کسانی که به شبکه NovaConnect متصل هستند، می‌توانند به داده‌های شما دسترسی داشته باشند.

نواستور

In [12]:
# Example 3: Different topic question
query3 = '''
سلام و عرض ادب
وقت شما بخیر
من یک سوالی داشتم ازتون
امکانش هست بفرمایید که در طرح Pro چند هسته پردازشی ارائه می‌شود؟
ممنون از شما
'''
print(f"User query: {query3}")

# Reset conversation history
conversation_history = []

# Time the response
start = time.time()
response = rag_chat(query3)
end = time.time()

# Stream or print the response
if stream_option:
    stream_text(response)
else:
    print(response)

print(f"---\nProcessing time: {end - start:.2f} seconds")

User query: 
سلام و عرض ادب
وقت شما بخیر
من یک سوالی داشتم ازتون
امکانش هست بفرمایید که در طرح Pro چند هسته پردازشی ارائه می‌شود؟
ممنون از شما

Similarity scores: [0.7008819108860892]
سلام و عرض ادب! وقت شما بخیر. خوشحالم که می‌توانم به سوالتان پاسخ دهم. در طرح حرفه‌ای (Pro)، ۸ هسته پردازشی ارائه می‌شود. این طرح با فضای ذخیره‌سازی ۵۰۰ گیگابایت و ترافیک نامحدود، برای کاربران حرفه‌ای و سازمانی مناسب است. اگر سوالی دیگر دارید، لطفا بپرسید.
---
Processing time: 2.68 seconds


In [27]:

query3 = '''
سلام و عرض ادب
وقت شما بخیر
من یک سوالی داشتم ازتون
امکانش هست بفرمایید که در طرح Pro چند هسته پردازشی ارائه می‌شود؟
ممنون از شما
'''
print(f"User query: {query3}")


"""Summarize query using the LLM"""
messages = [
    # {"role": "system", "content": "تو یک دستیار هستی که وظیفه تو خلاصه کردن متن است. سعی نکن به کاربر جواب بدی فقط تشریفات رو از بین ببر و تا جایی که امکان داره خلاصه کن. فقط تا جایی پرامپت کاربر رو کوتاه کن که به هسته اصلی مطلب آسیبی وارد نشه."},
    {"role": "system", "content": "خلاصه کن, خلاصه کن, خلاصه کن, خلاصه کن, خلاصه کن"},
    {"role": "user", "content": query3},
]

response = llm.create_chat_completion(
    messages=messages,
    top_p=0.85,
    temperature=0.1  # Low temperature for more deterministic responses
)
print(f"Summarized version: {response['choices'][0]['message']['content']}")


User query: 
سلام و عرض ادب
وقت شما بخیر
من یک سوالی داشتم ازتون
امکانش هست بفرمایید که در طرح Pro چند هسته پردازشی ارائه می‌شود؟
ممنون از شما

Summarized version: سلام و عرض ادب به شما! وقت شما بخیر. خوشحالم که می‌توانم به سوالتان پاسخ دهم.

در طرح Pro، 8 هسته پردازشی ارائه می‌شود. این تعداد هسته پردازشی به شما امکان می‌دهد که به راحتی کارهای چندگانه را انجام دهید و از عملکرد بهتر لذت ببرید. علاوه بر این، هسته‌های پردازشی در طرح Pro به شما امکان می‌دهند که از فناوری‌های پیشرفته مانند Turbo Boost و Hyper-Threading استفاده کنید که عملکرد پردازنده را بهبود می‌بخشد.


## 11. RAG System Evaluation

Test with more complex queries to evaluate retrieval performance and answer quality.

In [13]:
# Add evaluation functions here if needed
def evaluate_retrieval(test_queries, ground_truth_answers):
    """Simple evaluation for retrieval quality"""
    # TODO: Implement evaluation metrics
    pass